In [ ]:
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel("data/companies.xlsx")

In [3]:
df.head()

,Entity_Name,LEI,ISIN
0,Shell PLC,21380068P1DRHMJ8KU70,BRRDSABDR009
1,A Finkl & Sons Co,NaN,NaN
2,A. Finkl & Sons Corp,NaN,NaN
3,AEP Generating Co,NaN,NaN
4,AGL Loy Yang Pty Ltd,NaN,NaN


In [4]:
df.shape

(23446, 3)

In [5]:
df.columns = df.columns.str.lower()

In [6]:
GLEIF_API_URL = "https://api.gleif.org/api/v1/lei-records"

In [7]:
import re

def clean_company_name(name):
    # Remove common words and punctuation for comparison
    name = re.sub(r"\b(Co|Ltd|Corp|LLC|Company|Inc)\b", "", name, flags=re.IGNORECASE)
    name = re.sub(r"[^\w\s]", "", name)  # Remove punctuation
    name = name.strip()
    return name

In [ ]:
# # Function to find similar strings
# def find_similar(data, threshold=80):
#     similar_pairs = []
#     for i in range(len(data)):
#         for j in range(i + 1, len(data)):
#             if fuzz.ratio(data[i], data[j]) > threshold:
#                 similar_pairs.append((data[i], data[j]))
#     return similar_pairs

In [32]:
# # Find similar company names
# similar_names = find_similar(df['Entity_Name'].tolist())

# print("Similar Company Names:")
# for name_pair in similar_names:
#     print(name_pair)


In [9]:
df['cleaned_entity_name'] = df['entity_name'].apply(clean_company_name)

In [10]:
df_cleaned = df.drop_duplicates(subset=['cleaned_entity_name'])

In [14]:
df.shape[0] - df_cleaned.shape[0]

870

In [ ]:
df_cleaned = df_cleaned[['cleaned_entity_name', 'entity_name', 'lei']]

In [8]:
def get_lei(company_name):
    params = {
        "filter[entity.legalName]": company_name,  # Search by legal name
        "page[size]": 1  # Limit to 1 result
    }
    response = requests.get(GLEIF_API_URL, params=params)
    if response.status_code == 200:
        results = response.json().get('data', [])
        if results:
            # Extract the LEI from the first result
            return results[0]['id']
        else:
            return None
    else:
        print(f"Error: {response.status_code}")
        return None

In [ ]:
def process_in_batches(companies, batch_size=100):
    results = []
    for i in range(0, len(companies), batch_size):
        batch = companies[i:i+batch_size]
        for company in batch:
            lei = get_lei_cached(company)
            results.append((company, lei))
        print(f"Processed batch {i // batch_size + 1}. Waiting for next batch...")
        time.sleep(60)  # Wait time between batches, e.g., 1 minute
    return results


In [36]:
import asyncio, pandas as pd, re

In [19]:
fd = pd.read_excel("data/companies.xlsx")

In [20]:
fd.head(3)

,cleaned_entity_name,entity_name,lei,entity_type,industry_classification,n_employees
0,Shell PLC,Shell PLC,21380068P1DRHMJ8KU70,NaN,NaN,NaN
1,A Finkl Sons,A Finkl & Sons Co,NaN,public_subsidiary,mining & metals,136.0
2,AEP Generating,AEP Generating Co,NaN,NaN,NaN,NaN


In [22]:
def clean_company_name(name):
        
        # Remove common words and punctuation
        name = re.sub(r"\b(Co|Ltd|Corp|LLC|Company|Inc)\b", "", name, flags=re.IGNORECASE)
        name = re.sub(r"[^\w\s]", "", name)
        name = name.strip()
        return name

In [23]:
fd.columns = fd.columns.str.lower()
fd['cleaned_entity_name'] = fd['entity_name'].apply(clean_company_name)

In [ ]:
def clean_data(self):
        
        df = self.load_data()
        df.columns = df.columns.str.lower()

        df['cleaned_entity_name'] = df['entity_name'].apply(self.clean_company_name)
        df_duplicate = df.drop_duplicates(subset=['cleaned_entity_name'])
        df_cleaned = df_duplicate[['cleaned_entity_name', 'entity_name', 'lei']]
        
        return df_cleaned

In [25]:
fd_duplicate = fd.drop_duplicates(subset=['cleaned_entity_name'])


In [46]:
fd.head(20)

,cleaned_entity_name,entity_name,lei,entity_type,industry_classification,n_employees
0,Shell PLC,Shell PLC,21380068P1DRHMJ8KU70,NaN,NaN,NaN
1,A Finkl Sons,A Finkl & Sons Co,NaN,public_subsidiary,mining & metals,136.0
2,AEP Generating,AEP Generating Co,NaN,NaN,NaN,NaN
3,AGL Loy Yang Pty,AGL Loy Yang Pty Ltd,5493000FMBSFUZ80YU09,private,utilities,48.0
4,AGL SA Generation Pty,AGL SA Generation Pty Ltd,NaN,public,utilities,3677.0
5,APR Energy,APR Energy Ltd,5493000GLL7CZRYG3481,private,oil & energy,338.0
6,AVIC IndustryFinance Holdings,AVIC Industry-Finance Holdings Co Ltd,NaN,NaN,NaN,NaN
7,AVIC Securities,AVIC Securities Co Ltd,NaN,NaN,NaN,NaN
8,Abinski Elektrometallurgicheski Zavod,Abinski Elektrometallurgicheski Zavod LLC,NaN,NaN,NaN,NaN
9,Abinski Elektrometallurgicheski Zavod OOO,Abinski Elektrometallurgicheski Zavod OOO,NaN,NaN,NaN,NaN


In [ ]:
def enrich_dataframe(self, df: pd.DataFrame):

        if not isinstance(df, pd.DataFrame):
            raise ValueError("Input must be a pandas DataFrame")
        
        if 'entity_type' in df.columns:
            companies_to_process = df[df['entity_type'].isnull()]
            if companies_to_process.empty:
                 logging.info("All companies already have the necessary fields populated")
                 return df
        else:
            logging.warning("'entity_type' column does not exist in the DataFrame.")
            companies_to_process = df

        
        # Get unique company names and their index in the dataframe
        company_names = companies_to_process['entity_name'].tolist()
        
        # Fetch data concurrently
        results = asyncio.run(self.fetch_all_company_info(company_names))

        if len(results) != len(company_names):
            raise ValueError("Results count doesn't match the number of company names.")
        
        # Create new columns in the DataFrame and populate with fetched data
        df['entity_type'] = [result['entity_type'] for result in results]
        df['industry_classification'] = [result['industry_classification'] for result in results]
        df['n_employees'] = [result['n_employees'] for result in results]

        return df